In [63]:
%load_ext autoreload

%autoreload 2

import numpy as np
import pandas as pd
import json
import utils
import itertools

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
config_df = pd.DataFrame(utils.get_config_dicts())
base_models = sorted(set(config_df["base_model"].values))
datasets = sorted(set(config_df["dataset"].values))
drop_scoring_base_bools = [False, True]
ensemble_configs = list(dict(zip(("base_model", "drop_scoring_base", "dataset"), p)) for p in itertools.product(base_models, drop_scoring_base_bools, datasets))

In [69]:
def multi_step(df_train, df_test):
    def cond_mean(scores, thresholds):
        i = 0
        while i < len(thresholds) and scores[i] < thresholds[i]:
            i+=1
        return np.mean(scores[:(i+1)])
    
    df_test["ms"] = 0
    auc_scores = []
    for z in np.arange(0, 3, 0.1):
        thresholds = [np.mean(df_train.loc[df_train['y'] == 1, model]) - z*np.std(df_train.loc[df_train['y'] == 1, model]) for model in df_train.columns.values[:-2]]
        for i in range(df_train.shape[0]):
            df_train.loc[i, "ms"] = cond_mean(df_train.iloc[i, :df_train.shape[1]-2].to_list(), thresholds)
        auc_scores.append(roc_auc_score(df_train["y"], df_train["ms"]))
    
    z_star = 0.1*np.argmax(auc_scores)
    thresholds = [np.mean(df_train.loc[df_train['y'] == 1, model]) - z_star*np.std(df_train.loc[df_train['y'] == 1, model]) for model in df_train.columns.values[:-2]]
    for i in range(df_test.shape[0]):
        df_test.loc[i, "ms"] = cond_mean(df_test.iloc[i, :df_test.shape[1]-2].to_list(), thresholds)
    return df_test["ms"]

def get_aucs(**kwargs):
    df = utils.get_pred_df(**kwargs)
    
    y = df["y"]
    X = df.drop("y", axis=1).values

    auc_dict = {}
    
    for model in df.columns.values[:-1]:
        auc_dict[model] = roc_auc_score(y, df.loc[:, model])

    auc_dict["max"] = roc_auc_score(y, X.max(axis=1))
    auc_dict["mean"] =  roc_auc_score(y, X.mean(axis=1))
    auc_dict["median"] = roc_auc_score(y, np.median(X, axis=1))

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

    model_lr = LogisticRegression()
    model_lr.fit(X_train, y_train)
    auc_dict["lr"] = roc_auc_score(y_test, model_lr.predict_proba(X_test)[:, 1])

    model_rf = RandomForestClassifier()
    model_rf.fit(X_train, y_train)
    auc_dict["rf"] = roc_auc_score(y_test, model_rf.predict_proba(X_test)[:, 1])
    
    model_nb = GaussianNB()
    model_nb.fit(X_train, y_train)
    auc_dict["nb"] = roc_auc_score(y_test, model_nb.predict_proba(X_test)[:, 1])
    
    model_sv = svm.SVC(probability=True)
    model_sv.fit(X_train, y_train)
    auc_dict["sv"] = roc_auc_score(y_test, model_sv.predict_proba(X_test)[:, 1])
    
    model_list = ["gpt2-medium", "EleutherAI-gpt-neo-125m", "roberta-base", "gpt2", "bert-base-cased", "y"] #355, 125, 125, 124, 110
    df_train = pd.concat([pd.DataFrame(X_train, columns=df.columns.values[:-1]), pd.DataFrame(np.array(y_train), columns=["y"])], axis=1)
    df_test = pd.concat([pd.DataFrame(X_test, columns=df.columns.values[:-1]), pd.DataFrame(np.array(y_test), columns=["y"])], axis=1)
    df_train = df_train[[model for model in model_list if model in df_train.columns.values]]
    df_test = df_test[[model for model in model_list if model in df_train.columns.values]]
    auc_dict["ms"] = roc_auc_score(y_test, multi_step(df_train, df_test))
    
    return auc_dict

In [70]:
df_ensemble_results_nested = pd.DataFrame([{**c, "aucs":get_aucs(**c)} for c in ensemble_configs])
df_ensemble_results = df_ensemble_results_nested.join(df_ensemble_results_nested["aucs"].apply(pd.Series)).drop("aucs", axis=1)
print(df_ensemble_results)
print(np.mean(df_ensemble_results[df_ensemble_results["drop_scoring_base"]==True], axis=0))

                 base_model  drop_scoring_base dataset  \
0   EleutherAI-gpt-neo-125m              False  german   
1   EleutherAI-gpt-neo-125m              False    xsum   
2   EleutherAI-gpt-neo-125m               True  german   
3   EleutherAI-gpt-neo-125m               True    xsum   
4                      gpt2              False  german   
5                      gpt2              False    xsum   
6                      gpt2               True  german   
7                      gpt2               True    xsum   
8               gpt2-medium              False  german   
9               gpt2-medium              False    xsum   
10              gpt2-medium               True  german   
11              gpt2-medium               True    xsum   

    EleutherAI-gpt-neo-125m  bert-base-cased  gpt2-medium      gpt2  \
0                  0.953675         0.369700     0.775925  0.823425   
1                  0.998200         0.628175     0.247875  0.696050   
2                       NaN     

/home/jcyskr/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


In [72]:
df_ensemble_results

,base_model,drop_scoring_base,dataset,EleutherAI-gpt-neo-125m,bert-base-cased,gpt2-medium,gpt2,roberta-base,max,mean,median,lr,rf,nb,sv,ms
0,EleutherAI-gpt-neo-125m,False,german,0.953675,0.369700,0.775925,0.823425,0.465750,0.948550,0.891050,0.780375,0.951970,0.975164,0.923645,0.967570,0.818555
1,EleutherAI-gpt-neo-125m,False,xsum,0.998200,0.628175,0.247875,0.696050,0.840350,0.783300,0.922950,0.846525,1.000000,1.000000,1.000000,1.000000,0.425287
2,EleutherAI-gpt-neo-125m,True,german,NaN,0.369700,0.775925,0.823425,0.465750,0.633600,0.730225,0.765425,0.780378,0.801314,0.773399,0.816913,0.741379
3,EleutherAI-gpt-neo-125m,True,xsum,NaN,0.628175,0.247875,0.696050,0.840350,0.318825,0.721425,0.739750,0.981117,0.974343,0.945402,0.981527,0.360427
4,gpt2,False,german,0.281400,0.274925,0.836850,0.988675,0.558500,0.911950,0.825625,0.719425,0.999589,0.999179,0.995895,1.000000,0.827997
5,gpt2,False,xsum,0.932550,0.563750,0.536425,0.978875,0.757200,0.817600,0.946425,0.933750,1.000000,1.000000,0.997126,1.000000,0.676108
6,gpt2,True,german,0.281400,0.274925,0.836850,NaN,0.558500,0.545200,0.434125,0.452700,0.934319,0.921182,0.899015,0.921182,0.827997
7,gpt2,True,xsum,0.932550,0.563750,0.536425,NaN,0.757200,0.557825,0.884550,0.910900,0.996305,0.987890,0.982759,0.995074,0.671593
8,gpt2-medium,False,german,0.305825,0.283900,0.979325,0.966300,0.594575,0.930225,0.912325,0.811975,0.991790,0.989122,0.980706,0.983580,0.974138
9,gpt2-medium,False,xsum,0.899875,0.553750,0.925200,0.901775,0.648475,0.925200,0.956850,0.906950,0.969622,0.954228,0.935550,0.970854,0.947455
